# KBMOD Results and Filtering  
  
This notebook demonstrates the basic functionality for loading and filtering results. KBMOD provides the ability to load results into a ``Results`` data structure and then apply a sequence of filters to those results.

# Setup
Before importing, make sure you have installed kbmod using `pip install .` in the root directory.  Also be sure you are running with python3 and using the correct notebook kernel.

In [ ]:
# everything we will need for this demo
from kbmod.results import Results
import matplotlib.pyplot as plt
import numpy as np

# Load the results

We use the fake result data provided in ``data/fake_results_noisy`` which is generated from 256 x 256 images with multiple fake objects inserted. KBMOD is run with wider than normal filter parameters so as to produce a noisy set of results.

The `Results` object behaves like an astropy Table with some additional book keeping to help with filtering.

In [ ]:
results = Results.from_trajectory_file("../data/fake_results_noisy/results_DEMO.txt")
print(f"Loaded {len(results)} results with columns {results.colnames}")

# Turn on filtered result tracking.
results.track_filtered = True

# Show the top 5 rows
print(results[0:5])

We can access individual rows and columns using the `[]` notation:

In [ ]:
results["likelihood"][0]

# Sorting Results

We can sort the results by any of the cols of a ``Results`` in either increasing or decreasing order by operating directly on its table object.

In [ ]:
results.table.sort(keys="obs_count")
print(f"Top 5 by observation count:")
print(results[0:5])

print(f"\nBottom 5 by Flux:")
results.table.sort(keys=["flux"], reverse=False)
print(results[0:5])

# Return to sorted by decreasing likelihood.
results.table.sort(keys=["likelihood"])

# Extracting Individual Attributes

Since the `Results`class stores data as a table, the user can easily extract all of the values for a given attribute of the results. For example we could extract all of the flux values and create a histogram.

In [ ]:
plt.hist(results["flux"])

# Filtering

Using the `filter_rows()` method, you can filter out individual rows based on either their indices or a Boolean mask. In addition to the indices/mask, the `filter_rows()` method allows you to specify and optional label for later analysis.

In [ ]:
# Filter out all results that have a likelihood < 40.0.
mask = results["likelihood"] > 40.0
results.filter_rows(mask, "likelihood")
print(f"{len(results)} results remaining.")

We can look at the rows that passed the filter.

In [ ]:
print(results)

Because we set ``results.track_filtered = True`` above, the ``Results`` object also keeps each row that was rejected by one of the filters. These rows are indexed by the filter name, allowing the user to determine which rows were removed during which filtering stage. 

We can use the ``get_filtered`` function to retrieve all the filtered rows for a given filter name:

In [ ]:
# Extract the rows that did not pass filter1.
filtered = results.get_filtered("likelihood")
print(filtered)

We can apply multiple filters to the ``Results`` object to progressively rule out more and more candidate trajectories. We can even apply the same filter with different parameters.

Next we filter out anything with fewer than 10 observations:

In [ ]:
# Filter out all results with fewer than 10 observations.
results.filter_rows(results["obs_count"] >= 10, "obscount=10")
print(f"{len(results)} results remaining.")

### Reverting filters

As long as we have ``track_filtered`` turned on, we can undo any of the filtering steps. This appends the previously filtered results to the end of the list (and thus does not preserve ordering). However we can always re-sort if needed.

In [ ]:
results.revert_filter("likelihood")
print(f"{len(results)} results remaining.")